In [87]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import cycle, islice
import datetime as datetime
import glob
from matplotlib import pyplot
import plotly.graph_objects as go
import plotly.io as pio
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss, make_scorer
import category_encoders as ce
from sklearn.ensemble import RandomForestRegressor
import warnings

%matplotlib inline

plt.style.use('default')
sns.set(style="whitegrid")
pd.options.display.float_format = '{:20,.2f}'.format
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (15, 5)

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from catboost import CatBoostRegressor

In [88]:
# for preprocessing the data
from sklearn.preprocessing import StandardScaler
# the model
from sklearn.ensemble import RandomForestClassifier
# for combining the preprocess with model training
from sklearn.pipeline import make_pipeline
# for optimizing the hyperparameters of the pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

In [89]:
train_labels = pd.read_csv('training_train_labels.csv',low_memory=False, dtype= {
    'damage_grade':'uint8'
}).set_index('building_id')
train_values = pd.read_csv('training_train_values.csv',low_memory=False, dtype= {
    'geo_level_1_id':'category', 
    'geo_level_2_id':'category',
    'geo_level_3_id':'category', 
    'count_floors_pre_eq':'uint8',
    'age':'uint16',
    'area_percentage':'uint16', 
    'height_percentage':'uint16', 
    'land_surface_condition':'category', 
    'foundation_type':'category',
    'roof_type':'category',
    'ground_floor_type':'category',
    'other_floor_type':'category',
    'position':'category',
    'plan_configuration':'category', 
    'has_superstructure_adobe_mud':'uint8',
    'has_superstructure_mud_mortar_stone':'uint8',
    'has_superstructure_stone_flag':'uint8',
    'has_superstructure_cement_mortar_stone':'uint8', 
    'has_superstructure_mud_mortar_brick':'uint8', 
    'has_superstructure_cement_mortar_brick':'uint8', 
    'has_superstructure_timber':'uint8', 
    'has_superstructure_bamboo':'uint8',
    'has_superstructure_rc_non_engineered':'uint8',
    'has_superstructure_rc_engineered':'uint8',
    'has_superstructure_other':'uint8', 
    'legal_ownership_status':'category',
    'count_families':'uint16', 
    'has_secondary_use':'uint8', 
    'has_secondary_use_agriculture':'uint8', 
    'has_secondary_use_hotel':'uint8',
    'has_secondary_use_rental':'uint8',
    'has_secondary_use_institution':'uint8',
    'has_secondary_use_school':'uint8', 
    'has_secondary_use_industry':'uint8', 
    'has_secondary_use_health_post':'uint8', 
    'has_secondary_use_gov_office':'uint8', 
    'has_secondary_use_use_police':'uint8', 
    'has_secondary_use_other':'uint8',
}).set_index('building_id')

pd.options.display.float_format = '{:20,.2f}'.format

In [90]:
train_values.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,0
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,0


In [91]:
train = train_values.merge(train_labels, on='building_id')
train

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,2
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,2
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007913,10,1382,11935,2,60,11,6,t,u,x,...,0,0,0,0,0,0,0,0,0,2
349181,13,1154,1044,2,0,8,4,n,r,n,...,0,0,0,0,0,0,0,0,0,1
1009561,26,1001,11008,1,5,15,3,t,i,x,...,0,0,0,0,0,0,0,0,0,2


In [92]:
train = train.drop('geo_level_1_id', axis=1)
train = train.drop('geo_level_2_id', axis=1)
train = train.drop('geo_level_3_id', axis=1)
train = train.drop('land_surface_condition', axis=1)
#train = train.drop('count_floors_pre_eq', axis=1)

In [93]:
#edad promedio segun cantidad de pisos
train['edad_promedio'] = train.groupby('count_floors_pre_eq')['age'].transform('mean')

In [94]:
train.head()

,count_floors_pre_eq,age,area_percentage,height_percentage,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,...,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade,edad_promedio
building_id,,,,,,,,,,,,,,,,,,,,,
802906,2,30,6,5,r,n,f,q,t,d,...,0,0,0,0,0,0,0,0,3,25.82
28830,2,10,8,7,r,n,x,q,s,d,...,0,0,0,0,0,0,0,0,2,25.82
94947,2,10,5,5,r,n,f,x,t,d,...,0,0,0,0,0,0,0,0,3,25.82
590882,2,10,6,5,r,n,f,x,s,d,...,0,0,0,0,0,0,0,0,2,25.82
201944,3,30,8,9,r,n,f,x,s,d,...,0,0,0,0,0,0,0,0,3,30.18


In [95]:
#cantidad de tipos de techo segun edad
train['tipo_techo'] = train.groupby('age')['roof_type'].transform('count')
#train = train.drop('roof_type', axis=1)

In [96]:
train.head()

,count_floors_pre_eq,age,area_percentage,height_percentage,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,...,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade,edad_promedio,tipo_techo
building_id,,,,,,,,,,,,,,,,,,,,,
802906,2,30,6,5,r,n,f,q,t,d,...,0,0,0,0,0,0,0,3,25.82,9299
28830,2,10,8,7,r,n,x,q,s,d,...,0,0,0,0,0,0,0,2,25.82,19889
94947,2,10,5,5,r,n,f,x,t,d,...,0,0,0,0,0,0,0,3,25.82,19889
590882,2,10,6,5,r,n,f,x,s,d,...,0,0,0,0,0,0,0,2,25.82,19889
201944,3,30,8,9,r,n,f,x,s,d,...,0,0,0,0,0,0,0,3,30.18,9299


In [97]:
#cantidad de familias por edificacion
train['flias_afectadas'] = train.groupby('building_id')['count_families'].transform('count')

In [98]:
train.head()

,count_floors_pre_eq,age,area_percentage,height_percentage,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,...,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade,edad_promedio,tipo_techo,flias_afectadas
building_id,,,,,,,,,,,,,,,,,,,,,
802906,2,30,6,5,r,n,f,q,t,d,...,0,0,0,0,0,0,3,25.82,9299,1
28830,2,10,8,7,r,n,x,q,s,d,...,0,0,0,0,0,0,2,25.82,19889,1
94947,2,10,5,5,r,n,f,x,t,d,...,0,0,0,0,0,0,3,25.82,19889,1
590882,2,10,6,5,r,n,f,x,s,d,...,0,0,0,0,0,0,2,25.82,19889,1
201944,3,30,8,9,r,n,f,x,s,d,...,0,0,0,0,0,0,3,30.18,9299,1


In [99]:
train = train.drop('roof_type', axis=1)
train = train.drop('other_floor_type', axis=1)
train = train.drop('position', axis=1)
#train = train.drop('other_floor_type', axis=1)
train = train.drop('plan_configuration', axis=1)
train = train.drop('legal_ownership_status', axis=1)
train = train.drop('ground_floor_type', axis=1)
train = train.drop('foundation_type', axis=1)

In [100]:
train_value = train['damage_grade']
train_data = train.iloc[:, train.columns != 'damage_grade']

In [101]:
train_data

,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,...,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,edad_promedio,tipo_techo,flias_afectadas
building_id,,,,,,,,,,,,,,,,,,,,,
802906,2,30,6,5,1,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,9299,1
28830,2,10,8,7,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,19889,1
94947,2,10,5,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,19889,1
590882,2,10,6,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,19889,1
201944,3,30,8,9,1,0,0,0,0,0,...,0,0,0,0,0,0,0,30.18,9299,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007913,2,60,11,6,0,0,0,0,0,1,...,0,0,0,0,0,0,0,25.82,1809,1
349181,2,0,8,4,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,13307,1
1009561,1,5,15,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,17.64,17361,1


In [102]:
test_labels = pd.read_csv('test_train_labels.csv',low_memory=False, dtype= {
    'damage_grade':'uint8'
}).set_index('building_id')
test_values = pd.read_csv('test_train_values.csv',low_memory=False, dtype= {
    'geo_level_1_id':'category', 
    'geo_level_2_id':'category',
    'geo_level_3_id':'category', 
    'count_floors_pre_eq':'uint8',
    'age':'uint16',
    'area_percentage':'uint16', 
    'height_percentage':'uint16', 
    'land_surface_condition':'category', 
    'foundation_type':'category',
    'roof_type':'category',
    'ground_floor_type':'category',
    'other_floor_type':'category',
    'position':'category',
    'plan_configuration':'category', 
    'has_superstructure_adobe_mud':'uint8',
    'has_superstructure_mud_mortar_stone':'uint8',
    'has_superstructure_stone_flag':'uint8',
    'has_superstructure_cement_mortar_stone':'uint8', 
    'has_superstructure_mud_mortar_brick':'uint8', 
    'has_superstructure_cement_mortar_brick':'uint8', 
    'has_superstructure_timber':'uint8', 
    'has_superstructure_bamboo':'uint8',
    'has_superstructure_rc_non_engineered':'uint8',
    'has_superstructure_rc_engineered':'uint8',
    'has_superstructure_other':'uint8', 
    'legal_ownership_status':'category',
    'count_families':'uint16', 
    'has_secondary_use':'uint8', 
    'has_secondary_use_agriculture':'uint8', 
    'has_secondary_use_hotel':'uint8',
    'has_secondary_use_rental':'uint8',
    'has_secondary_use_institution':'uint8',
    'has_secondary_use_school':'uint8', 
    'has_secondary_use_industry':'uint8', 
    'has_secondary_use_health_post':'uint8', 
    'has_secondary_use_gov_office':'uint8', 
    'has_secondary_use_use_police':'uint8', 
    'has_secondary_use_other':'uint8',
}).set_index('building_id')

pd.options.display.float_format = '{:20,.2f}'.format

In [103]:
test = test_values.merge(test_labels, on='building_id')

In [104]:
test = test.drop('geo_level_1_id', axis=1)
test = test.drop('geo_level_2_id', axis=1)
test = test.drop('geo_level_3_id', axis=1)
test = test.drop('land_surface_condition', axis=1)

In [105]:
#edad promedio segun cantidad de pisos
test['edad_promedio'] = test.groupby('count_floors_pre_eq')['age'].transform('mean')

In [106]:
#cantidad de tipos de techo segun edad
test['tipo_techo'] = test.groupby('age')['roof_type'].transform('count')

In [107]:
#cantidad de familias por edificacion
test['cantidad_flias'] = test.groupby('building_id')['count_families'].transform('count')

In [108]:
test = test.drop('roof_type', axis=1)
test = test.drop('other_floor_type', axis=1)
test = test.drop('position', axis=1)
#test = test.drop('other_floor_type', axis=1)
test = test.drop('plan_configuration', axis=1)
test = test.drop('legal_ownership_status', axis=1)
test = test.drop('ground_floor_type', axis=1)
test = test.drop('foundation_type', axis=1)


#test = test.drop_duplicates('damage_grade')
IDs = test[['damage_grade']].copy()
#test = test.drop('damage_grade', axis=1)

In [109]:
test.head()

,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,...,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade,edad_promedio,tipo_techo,cantidad_flias
building_id,,,,,,,,,,,,,,,,,,,,,
1045849,2,10,5,5,0,1,0,0,0,0,...,0,0,0,0,0,0,2,26.61,6255,1
523230,2,15,11,5,0,1,0,0,0,0,...,0,0,0,0,0,0,3,26.61,5750,1
183337,4,80,9,10,0,0,0,0,0,1,...,0,0,0,0,0,0,2,59.25,482,1
1038657,3,40,9,7,0,1,0,0,0,0,...,0,0,0,0,0,0,2,30.07,1715,1
237062,2,15,3,4,0,1,0,0,0,0,...,0,0,0,0,0,0,2,26.61,5750,1


In [110]:
test_value = test['damage_grade']
test_data = test.iloc[:, test.columns != 'damage_grade']

In [111]:
test.head()

,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,...,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade,edad_promedio,tipo_techo,cantidad_flias
building_id,,,,,,,,,,,,,,,,,,,,,
1045849,2,10,5,5,0,1,0,0,0,0,...,0,0,0,0,0,0,2,26.61,6255,1
523230,2,15,11,5,0,1,0,0,0,0,...,0,0,0,0,0,0,3,26.61,5750,1
183337,4,80,9,10,0,0,0,0,0,1,...,0,0,0,0,0,0,2,59.25,482,1
1038657,3,40,9,7,0,1,0,0,0,0,...,0,0,0,0,0,0,2,30.07,1715,1
237062,2,15,3,4,0,1,0,0,0,0,...,0,0,0,0,0,0,2,26.61,5750,1


In [112]:
test_data.head()

,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,...,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,edad_promedio,tipo_techo,cantidad_flias
building_id,,,,,,,,,,,,,,,,,,,,,
1045849,2,10,5,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,26.61,6255,1
523230,2,15,11,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,26.61,5750,1
183337,4,80,9,10,0,0,0,0,0,1,...,0,0,0,0,0,0,0,59.25,482,1
1038657,3,40,9,7,0,1,0,0,0,0,...,0,0,0,0,0,0,0,30.07,1715,1
237062,2,15,3,4,0,1,0,0,0,0,...,0,0,0,0,0,0,0,26.61,5750,1


In [113]:
LogLoss = make_scorer(log_loss, greater_is_better=False, needs_proba=False)

In [114]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(train_data, train_value, train_size=0.7, random_state=1234)

In [115]:
train_data

,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,...,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,edad_promedio,tipo_techo,flias_afectadas
building_id,,,,,,,,,,,,,,,,,,,,,
802906,2,30,6,5,1,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,9299,1
28830,2,10,8,7,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,19889,1
94947,2,10,5,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,19889,1
590882,2,10,6,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,19889,1
201944,3,30,8,9,1,0,0,0,0,0,...,0,0,0,0,0,0,0,30.18,9299,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007913,2,60,11,6,0,0,0,0,0,1,...,0,0,0,0,0,0,0,25.82,1809,1
349181,2,0,8,4,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,13307,1
1009561,1,5,15,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,17.64,17361,1


In [116]:
categorical_features_indices = np.where(train_data.dtypes == np.object)[0]

In [117]:
%%capture
modelCatBoost=CatBoostRegressor(iterations=250, depth=3, learning_rate=0.05)
modelCatBoost.fit(X_train, y_train,cat_features=categorical_features_indices,eval_set=(X_validation, y_validation),plot=True)

0:	learn: 0.6068625	test: 0.6087446	best: 0.6087446 (0)	total: 99.8ms	remaining: 24.9s
1:	learn: 0.6031348	test: 0.6050190	best: 0.6050190 (1)	total: 198ms	remaining: 24.5s
2:	learn: 0.5996545	test: 0.6015368	best: 0.6015368 (2)	total: 328ms	remaining: 27s
3:	learn: 0.5963590	test: 0.5981649	best: 0.5981649 (3)	total: 451ms	remaining: 27.7s
4:	learn: 0.5933727	test: 0.5951259	best: 0.5951259 (4)	total: 475ms	remaining: 23.3s
5:	learn: 0.5906645	test: 0.5923670	best: 0.5923670 (5)	total: 499ms	remaining: 20.3s
6:	learn: 0.5881384	test: 0.5897973	best: 0.5897973 (6)	total: 523ms	remaining: 18.2s
7:	learn: 0.5858579	test: 0.5875215	best: 0.5875215 (7)	total: 584ms	remaining: 17.7s
8:	learn: 0.5838111	test: 0.5854740	best: 0.5854740 (8)	total: 609ms	remaining: 16.3s
9:	learn: 0.5818068	test: 0.5834291	best: 0.5834291 (9)	total: 636ms	remaining: 15.3s
10:	learn: 0.5800783	test: 0.5816990	best: 0.5816990 (10)	total: 660ms	remaining: 14.3s
11:	learn: 0.5785273	test: 0.5801190	best: 0.5801190 

98:	learn: 0.5510675	test: 0.5526897	best: 0.5526897 (98)	total: 5.08s	remaining: 7.75s
99:	learn: 0.5510200	test: 0.5526546	best: 0.5526546 (99)	total: 5.15s	remaining: 7.72s
100:	learn: 0.5509737	test: 0.5526086	best: 0.5526086 (100)	total: 5.22s	remaining: 7.71s
101:	learn: 0.5509196	test: 0.5525642	best: 0.5525642 (101)	total: 5.34s	remaining: 7.76s
102:	learn: 0.5508492	test: 0.5525155	best: 0.5525155 (102)	total: 5.38s	remaining: 7.67s
103:	learn: 0.5508107	test: 0.5524795	best: 0.5524795 (103)	total: 5.4s	remaining: 7.58s
104:	learn: 0.5507471	test: 0.5524376	best: 0.5524376 (104)	total: 5.42s	remaining: 7.49s
105:	learn: 0.5507007	test: 0.5523943	best: 0.5523943 (105)	total: 5.44s	remaining: 7.4s
106:	learn: 0.5506587	test: 0.5523496	best: 0.5523496 (106)	total: 5.47s	remaining: 7.31s
107:	learn: 0.5506106	test: 0.5523090	best: 0.5523090 (107)	total: 5.49s	remaining: 7.22s
108:	learn: 0.5505746	test: 0.5522814	best: 0.5522814 (108)	total: 5.52s	remaining: 7.14s
109:	learn: 0.55

192:	learn: 0.5483110	test: 0.5502592	best: 0.5502592 (192)	total: 8.99s	remaining: 2.66s
193:	learn: 0.5482828	test: 0.5502386	best: 0.5502386 (193)	total: 9.04s	remaining: 2.61s
194:	learn: 0.5482635	test: 0.5502315	best: 0.5502315 (194)	total: 9.06s	remaining: 2.56s
195:	learn: 0.5482534	test: 0.5502253	best: 0.5502253 (195)	total: 9.08s	remaining: 2.5s
196:	learn: 0.5482431	test: 0.5502171	best: 0.5502171 (196)	total: 9.12s	remaining: 2.45s
197:	learn: 0.5482288	test: 0.5502048	best: 0.5502048 (197)	total: 9.27s	remaining: 2.44s
198:	learn: 0.5482183	test: 0.5502020	best: 0.5502020 (198)	total: 9.36s	remaining: 2.4s
199:	learn: 0.5482052	test: 0.5501934	best: 0.5501934 (199)	total: 9.42s	remaining: 2.35s
200:	learn: 0.5481948	test: 0.5501842	best: 0.5501842 (200)	total: 9.45s	remaining: 2.3s
201:	learn: 0.5481880	test: 0.5501743	best: 0.5501743 (201)	total: 9.5s	remaining: 2.26s
202:	learn: 0.5481809	test: 0.5501730	best: 0.5501730 (202)	total: 9.53s	remaining: 2.21s
203:	learn: 0.

In [149]:
preds = modelCatBoost.predict(X_train)
logloss = log_loss(y_train.replace(3, 1), preds)     #tener solo 2 opciones - REVISAR
print("logloss: %f" % (logloss))

logloss: 14.854712


In [ ]:
pd.to_numeric(preds.round(), downcast='signed')

In [132]:
y_train.replace(3, 1)

building_id
323246     2
666127     1
990198     2
934274     2
490051     2
          ..
847186     1
685157     2
741415     1
894354     2
1018556    1
Name: damage_grade, Length: 93553, dtype: uint8

In [ ]:
type(preds)

In [136]:
%%capture
modelCatBoost.fit(train_data,train_value,cat_features=categorical_features_indices)
preds = modelCatBoost.predict(test)
entregable_CatBoost = IDs[['damage_grade']].copy()
entregable_CatBoost['Target'] = preds

0:	learn: 0.6074042	total: 72ms	remaining: 17.9s
1:	learn: 0.6035335	total: 111ms	remaining: 13.7s
2:	learn: 0.6000186	total: 148ms	remaining: 12.2s
3:	learn: 0.5968143	total: 182ms	remaining: 11.2s
4:	learn: 0.5939748	total: 217ms	remaining: 10.6s
5:	learn: 0.5912467	total: 249ms	remaining: 10.1s
6:	learn: 0.5888345	total: 337ms	remaining: 11.7s
7:	learn: 0.5865535	total: 381ms	remaining: 11.5s
8:	learn: 0.5844426	total: 425ms	remaining: 11.4s
9:	learn: 0.5823927	total: 468ms	remaining: 11.2s
10:	learn: 0.5806813	total: 509ms	remaining: 11.1s
11:	learn: 0.5789347	total: 579ms	remaining: 11.5s
12:	learn: 0.5772925	total: 620ms	remaining: 11.3s
13:	learn: 0.5758940	total: 651ms	remaining: 11s
14:	learn: 0.5746288	total: 688ms	remaining: 10.8s
15:	learn: 0.5733878	total: 723ms	remaining: 10.6s
16:	learn: 0.5722886	total: 774ms	remaining: 10.6s
17:	learn: 0.5711159	total: 931ms	remaining: 12s
18:	learn: 0.5700555	total: 984ms	remaining: 12s
19:	learn: 0.5690232	total: 1.02s	remaining: 11.

164:	learn: 0.5494346	total: 9.26s	remaining: 4.77s
165:	learn: 0.5494081	total: 9.32s	remaining: 4.72s
166:	learn: 0.5493732	total: 9.36s	remaining: 4.65s
167:	learn: 0.5493314	total: 9.4s	remaining: 4.59s
168:	learn: 0.5493149	total: 9.45s	remaining: 4.53s
169:	learn: 0.5492985	total: 9.61s	remaining: 4.52s
170:	learn: 0.5492536	total: 9.71s	remaining: 4.48s
171:	learn: 0.5492363	total: 9.75s	remaining: 4.42s
172:	learn: 0.5492256	total: 9.79s	remaining: 4.36s
173:	learn: 0.5492082	total: 9.94s	remaining: 4.34s
174:	learn: 0.5491763	total: 10s	remaining: 4.3s
175:	learn: 0.5491622	total: 10.1s	remaining: 4.24s
176:	learn: 0.5491492	total: 10.1s	remaining: 4.17s
177:	learn: 0.5491430	total: 10.2s	remaining: 4.13s
178:	learn: 0.5491099	total: 10.3s	remaining: 4.09s
179:	learn: 0.5490920	total: 10.4s	remaining: 4.04s
180:	learn: 0.5490833	total: 10.4s	remaining: 3.97s
181:	learn: 0.5490536	total: 10.5s	remaining: 3.91s
182:	learn: 0.5490134	total: 10.5s	remaining: 3.85s
183:	learn: 0.54

In [137]:
entregable_CatBoost.to_csv('CatBoost.csv', index=False)

In [138]:
modeloCB = CatBoostRegressor()
LogLoss = make_scorer(log_loss, greater_is_better = False)
categorical_features_indices = np.where(train_data.dtypes == np.object)[0]

In [141]:
parameters = {'iterations':[100, 250, 500] , 'depth':[2, 3, 5], 'learning_rate':[0.005, 0.05, 0.5]}
gscv = GridSearchCV(modeloCB, parameters, cv=5, scoring=LogLoss)

In [142]:
train_data

,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,...,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,edad_promedio,tipo_techo,flias_afectadas
building_id,,,,,,,,,,,,,,,,,,,,,
802906,2,30,6,5,1,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,9299,1
28830,2,10,8,7,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,19889,1
94947,2,10,5,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,19889,1
590882,2,10,6,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,19889,1
201944,3,30,8,9,1,0,0,0,0,0,...,0,0,0,0,0,0,0,30.18,9299,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007913,2,60,11,6,0,0,0,0,0,1,...,0,0,0,0,0,0,0,25.82,1809,1
349181,2,0,8,4,0,1,0,0,0,0,...,0,0,0,0,0,0,0,25.82,13307,1
1009561,1,5,15,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,17.64,17361,1


In [151]:
train_values = train_values.iloc[:133648]

In [152]:
train_values

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,0
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007913,10,1382,11935,2,60,11,6,t,u,x,...,0,0,0,0,0,0,0,0,0,0
349181,13,1154,1044,2,0,8,4,n,r,n,...,0,0,0,0,0,0,0,0,0,0
1009561,26,1001,11008,1,5,15,3,t,i,x,...,0,0,0,0,0,0,0,0,0,0


In [153]:
%%capture
gscv.fit(train_data, train_values, cat_features = categorical_features_indices)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already speci

Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already specified. Specify more than one logger at same time is not thread safe.Custom logger is already speci

CatBoostError: catboost/private/libs/target/data_providers.cpp:533: Currently only multi-regression and survival objectives work with multidimensional target

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

In [154]:
print(gscv.best_params_)

{'depth': 2, 'iterations': 100, 'learning_rate': 0.005}


In [157]:
%%capture
modeloCB = CatBoostRegressor(depth = 5, iterations = 100, learning_rate = 0.05)
modeloCB.fit(train_data, train_value, cat_features = categorical_features_indices)
result = modeloCB.predict(test)

entregable_CB = IDs[['damage_grade']].copy() #revisar
entregable_CB['Target'] = result #revisar

0:	learn: 0.6066649	total: 15.6ms	remaining: 1.54s
1:	learn: 0.6022365	total: 32.4ms	remaining: 1.59s
2:	learn: 0.5980752	total: 48.6ms	remaining: 1.57s
3:	learn: 0.5942718	total: 64.2ms	remaining: 1.54s
4:	learn: 0.5908039	total: 80.4ms	remaining: 1.53s
5:	learn: 0.5876481	total: 96ms	remaining: 1.5s
6:	learn: 0.5847817	total: 114ms	remaining: 1.51s
7:	learn: 0.5822441	total: 131ms	remaining: 1.5s
8:	learn: 0.5798911	total: 146ms	remaining: 1.48s
9:	learn: 0.5777652	total: 162ms	remaining: 1.46s
10:	learn: 0.5758224	total: 178ms	remaining: 1.44s
11:	learn: 0.5738011	total: 194ms	remaining: 1.42s
12:	learn: 0.5721527	total: 210ms	remaining: 1.41s
13:	learn: 0.5706226	total: 247ms	remaining: 1.52s
14:	learn: 0.5689638	total: 263ms	remaining: 1.49s
15:	learn: 0.5676843	total: 279ms	remaining: 1.47s
16:	learn: 0.5664625	total: 295ms	remaining: 1.44s
17:	learn: 0.5653921	total: 311ms	remaining: 1.41s
18:	learn: 0.5643840	total: 326ms	remaining: 1.39s
19:	learn: 0.5632746	total: 341ms	remai

In [159]:
entregable_CB.to_csv('CB_Final.csv', index=False)